## Training Loop

In [1]:
import lightning as L
import torch
from torch import nn, optim
import torch.nn.functional as F
import math
import pandas as pd
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

class LitIndexer(L.LightningModule):
    def __init__(self, mapper, indexer, name):
        super().__init__()
        self.name = name
        self.mapper = mapper
        self.indexer = indexer
        self.min_loss = float("inf")
        self.epoch_losses = []
        self.test_losses = []
        self.results_df = pd.DataFrame(columns=["Name", "Test Loss", "Percent Narrowed"])
        
        self.early_stop_callback = EarlyStopping(
            monitor='train_loss',
            min_delta=0.001,
            patience=3,
            verbose=True,
            mode='min'
        )
        
    def forward(self, strs):
        mapped_strs = self.mapper.forward(strs)
        pred_idxs = self.indexer(mapped_strs)
        return pred_idxs
        
    def loss(self, pred_idxs, real_idxs):
        return F.mse_loss(pred_idxs, real_idxs)
    
    def percent_narrowed(self, loss):
        return math.sqrt(loss) / len(self.mapper.data)

    def training_step(self, batch, batch_idx):
        strs, idxs = batch
        mapped_strs = self.mapper.forward(strs)
        pred_idxs = self.indexer(mapped_strs)
        loss = self.loss(pred_idxs, idxs.to(torch.float32))
        self.min_loss = min(self.min_loss, loss)
        self.epoch_losses.append(loss)
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def on_train_epoch_end(self):
        avg_epoch_loss = sum(self.epoch_losses) / len(self.epoch_losses)
        # print(f"Epoch {self.current_epoch}, Average Epoch Loss: {avg_epoch_loss:.4f}, Percent Narrowed: {self.percent_narrowed(avg_epoch_loss):.4f}")
        self.epoch_losses = []
    
    def test_step(self, batch, batch_idx):
        strs, idxs = batch
        mapped_strs = self.mapper.forward(strs)
        pred_idxs = self.indexer(mapped_strs)
        loss = self.loss(pred_idxs, idxs.to(torch.float32))
        self.test_losses.append(loss)
        self.log("test_loss", loss, prog_bar=True)
        
    def on_test_epoch_end(self):
        avg_test_loss = sum(self.test_losses) / len(self.test_losses)
        print(f"Average Test Loss: {avg_test_loss:.4f}, Percent Narrowed: {self.percent_narrowed(avg_test_loss):.4f}")
        self.test_losses = []
        self.results_df = pd.concat([self.results_df, pd.DataFrame({"Name": [self.name], 
                                                             "Test Loss": [avg_test_loss.item()], 
                                                             "Percent Narrowed": [self.percent_narrowed(avg_test_loss)]})], ignore_index=True)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-2)
        return optimizer


## Experiment Runner

In [2]:
from models.BaselineHash import BaselineHash
from models.BaselineEmbed import BaselineEmbed
from models.LinearModel import LinearRegressionModel
from models.TwoLinearModel import TwoLayerLinearRegressionModel
from IndexDataset import IndexDataset
from torch.utils.data import DataLoader
import mmh3

experiments = {
            # IMDB
            "linear-hash_imdb": {"dataset_name": "data/imdb/imdb_top_1000.csv", 
                               "column_name": "Overview",
                               "mapper": "BaselineHash", 
                               "indexer": "2LinearModel"},
            "linear-embed_imdb": {"dataset_name": "data/imdb/imdb_top_1000.csv", 
                               "column_name": "Overview",
                               "mapper": "BaselineEmbed", 
                               "indexer": "2LinearModel"},
            # LEGO
            "linear-hash_lego": {"dataset_name": "data/lego/inventory_parts.csv", 
                               "column_name": "part_num",
                               "mapper": "BaselineHash", 
                               "indexer": "2LinearModel"},
            "linear-embed_lego": {"dataset_name": "data/lego/inventory_parts.csv", 
                               "column_name": "part_num",
                               "mapper": "BaselineEmbed", 
                               "indexer": "2LinearModel"},
            
            # RANDOM 100
            "linear-hash_rand-100": {"dataset_name": "data/random/random_100.csv", 
                               "column_name": "RandomString",
                               "mapper": "BaselineHash", 
                               "indexer": "2LinearModel"},
            "linear-embed_rand-100": {"dataset_name": "data/random/random_100.csv", 
                               "column_name": "RandomString",
                               "mapper": "BaselineEmbed", 
                               "indexer": "2LinearModel"},
            # RANDOM 1000000
            "linear-hash_rand-1000000": {"dataset_name": "data/random/random_1000000.csv", 
                               "column_name": "RandomString",
                               "mapper": "BaselineHash", 
                               "indexer": "2LinearModel"},
            "linear-embed_rand-1000000": {"dataset_name": "data/random/random_1000000.csv", 
                               "column_name": "RandomString",
                               "mapper": "BaselineEmbed", 
                               "indexer": "2LinearModel"},
            }

In [3]:
from pytorch_lightning import loggers as pl_loggers

batch_size = 128
token_len = 1
embed_size = 1
hash = mmh3.hash    # 32-bits
num_minutes = 1
max_epochs = 100

results_df = pd.DataFrame(columns=["Name", "Test Loss", "Percent Narrowed"])

for name, experiment_dict in experiments.items():
    dataset_name, column_name, mapper_type, indexer_type = experiment_dict["dataset_name"], experiment_dict["column_name"], \
                                                            experiment_dict["mapper"], experiment_dict["indexer"]
    
    print("#################")
    print(f"{name}: {dataset_name}, {column_name}")
    
    df = pd.read_csv(dataset_name)
    dataset = IndexDataset(df, column_name)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    mapper, indexer = None, None
    vector_size = None
    if mapper_type == "BaselineHash":
        mapper = BaselineHash(dataset, hash)
        vector_size = 1
    elif mapper_type == "BaselineEmbed":
        mapper = BaselineEmbed(dataset, token_len, embed_size)
        vector_size = mapper.max_len * embed_size
    
    if indexer_type == "LinearModel":
        indexer = LinearRegressionModel(vector_size)
    elif indexer_type == "2LinearModel":
        indexer = TwoLayerLinearRegressionModel(vector_size)
    
    model = LitIndexer(mapper, indexer, name)
    
    tb_logger = pl_loggers.TensorBoardLogger('lightning_logs/')
    trainer = L.Trainer(accelerator="cpu", logger=tb_logger,
                        max_epochs=max_epochs,
                        max_time={"minutes": num_minutes}
                        )
    
    trainer.fit(model, train_dataloaders=dataloader)
    
    torch.save(model.state_dict(), f"models/{name}.pth")
    
    trainer.test(model, dataloaders=dataloader)
    
    results_df = pd.concat([results_df, model.results_df], ignore_index=True)

results_df

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


#################
linear-hash_imdb: data/imdb/imdb_top_1000.csv, Overview


/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

  | Name    | Type                          | Params
----------------------------------------------------------
0 | indexer | TwoLayerLinearRegressionModel | 4     
----------------------------------------------------------
4         Trainable params
0         Non-trainable params
4         Total params
0.000     Total estimated model params size (MB)
/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/loops/fit_loop.py:298: Th

Training: |          | 0/? [00:00<?, ?it/s]

/var/folders/9c/5hbdnj_57xv861zqcz43bs6w0000gn/T/ipykernel_5943/620326400.py:34: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred_idxs, real_idxs)
`Trainer.fit` stopped: `max_epochs=100` reached.
/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/alexanderkumar/miniconda3/envs/graphs/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


...

Testing: |          | 0/? [00:00<?, ?it/s]

Average Test Loss: 83333668864.0000, Percent Narrowed: 0.2887


/var/folders/9c/5hbdnj_57xv861zqcz43bs6w0000gn/T/ipykernel_5943/620326400.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results_df = pd.concat([self.results_df, pd.DataFrame({"Name": [self.name],


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       83333611520.0       │
└───────────────────────────┴───────────────────────────┘

,Name,Test Loss,Percent Narrowed
0,linear-hash_imdb,5.774958e+04,0.375487
1,linear-embed_imdb,8.501633e+04,0.291576
2,linear-hash_lego,2.825306e+07,0.362848
3,linear-embed_lego,4.462968e+07,0.288813
4,linear-hash_rand-100,1.220565e+03,0.554549
5,linear-embed_rand-100,1.017619e+03,0.319001
6,linear-hash_rand-1000000,1.329994e+11,0.577104
7,linear-embed_rand-1000000,8.333367e+10,0.288676
